In [ ]:
%%HTML
<p id = 'heading1_p'> WPC Authentication – ECC204-TFLXWPC</p>
<hr id = 'hline'> </hr> 
<p id = 'heading1_p' style='font-size: 20px; font-weight: bold;'>Introduction</p>
<div id='div_element'>
<p>
    The purpose of this Use Case is to authentify a Qi 1.3 Wireless Power Charger from a mobile phone, 
    as Qi 1.3 specification mandate the usage of a Secure subsystem (Secure Element) on the charger side. 
    It is based on a standard asymmetric authentication
</p>
<p>
    Asymmetric Authentication is a process based on a custom PKI (Public Key Infrastructure) where a Host (mobile phone) 
    will authenticate that the WPC charger (device) is genuine. The Host will first verify the Signer certificate (Manufacturer ) 
    device certificates (Product Unit Certificate – PUC) based on the Root CA Public key and will generate a challenge to be 
    signed by the charger private key. The Host will then perform an ECDSA verify command to ensure that the signed challenge is valid.
</p>
<p>
    This use case describes how Microchip ECC204 TrustFLEX WPC Secure Subsystem can be used for 
    Qi 1.3 WPC authentication using asymmetric authentication (Custom PKI based).
</p>
</div>

<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Cryptographic Asset list & Use Case implementation</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p style="float: left; font-size: 9pt; text-align: center; width: 30%; margin-right: 1%;">
    <img src="assets/images/wpc_auth/crypto_assets.png" alt="Cryptographic Assets" style="width: 100%"><b>Cryptographic Asset list</b></p>
<p style="float: left; font-size: 9pt; text-align: center; 20em; width: 40%; margin-right: 1%;">
    <img src="assets/images/wpc_auth/usecase_steps.png" alt="Use Case Steps" style="height: 30em; width: 100%"><b>Use Case implementation</b></p>
<p style="clear: both;">
</div>
</div>

<p id = 'heading1_p' style='margin-top:40px; font-size: 20px; font-weight: bold;'>Use Case information</p>
<hr id = 'hline'> </hr> 
<div id='div_element'>
<p> The Transaction Diagram represent all steps to be performed to successfully implement the given Use Case, during development and after product delivery to the end-user. </p>
<p><strong> For the ease of the execution, when steps are executed from transaction diagram, these are emulated in the Host-PC rather than executed in the 
Host-Microcontroller/Processor from the evaluation kit.</strong> The MPLAB-X project/C Source code includes software code to be executed in your Host-Microcontroller/Processor 
in your embedded project. </p>

<p>Select your Evaluation kit and then click on each step to execute the Use Case. Click on Usecase Help button in transaction diagram for additional details.</p>
</div>

In [ ]:
import os
from tpds.usecase_diagram import ProtoBoard, UsecaseDiagram
from tpds.tp_utils.tp_settings import TPSettings
from tpds.proto_boards import get_board_path
from IPython.display import display
from pathlib import Path

curr_path = os.path.abspath('')
os.sys.path.extend([os.path.join(curr_path, 'assets', 'python')])
os.sys.path.extend([str(Path(curr_path).parents[2])])
import wpc_auth

tp_settings = TPSettings()
usecase_working_dir = os.path.join(tp_settings.get_base_folder(), 'spg_ecc204_wpc_auth')
os.makedirs(usecase_working_dir, exist_ok=True)
boards = ProtoBoard(max_width=400)
boards.add_board(
            'DM320118', 
            os.path.join(curr_path, 'assets', 'images', 'usecase_hw.png'),
            os.path.join(get_board_path(), 'DM320118', 'DM320118.hex'),
            os.path.join(curr_path, 'firmware', 'wpc_auth_firmware', 'wpc_auth_firmware.X'))
boards.render_boards(0)

wpc = wpc_auth.WpcAuthentication(boards)
usecase_td = UsecaseDiagram(os.path.join(curr_path, 'assets', 'images','wpc_auth', 'usecase_td.png'), working_dir=usecase_working_dir)
usecase_td.add_firmware(boards.get_firmware_project, tp_settings.get_mplab_paths().get('ide_path'))


generate_resources = usecase_td.add_script(130, 110, 35, 30, wpc.generate_resources, script_index='1')
read_certs = usecase_td.add_script(330, 225, 35, 30, wpc.read_certs_from_device, prereq_scripts=[generate_resources],script_index='2a')
verify_cert_chain = usecase_td.add_script(392, 256, 35, 30, wpc.verify_cert_chain, prereq_scripts=[read_certs], script_index='2b')
read_chain_digest = usecase_td.add_script(338, 295, 35, 30, wpc.read_wpc_chain_digest, prereq_scripts=[generate_resources], script_index='3a')
verify_chain_digest = usecase_td.add_script(392, 323, 35, 30, wpc.verify_wpc_chain_digest, prereq_scripts=[read_chain_digest], script_index = '3b')
send_challenge = usecase_td.add_script(330, 370, 35, 30, wpc.send_random_challenge_to_PT, prereq_scripts=[verify_chain_digest], script_index = '4')
verify_challenge = usecase_td.add_script(382, 402, 35, 30, wpc.verify_PT_response, prereq_scripts=[send_challenge], script_index = '5')

usecase_td.add_image(825, 201, 140, 25, os.path.join(curr_path, 'assets', 'images', 'wpc_auth', 'c_snippet.PNG'), 'Code Snippet')

usecase_td.add_usecase_help(os.path.join(curr_path, 'assets', 'ECC204_WPC_Auth.md'))

display(boards.canvas)
usecase_td.display_canvas()

In [ ]:
%%HTML
<button class="collapsible" style='font-size: 20px; font-weight: bold;'>Conclusion</button>
<div class="content">
<div id='content_element'>
<hr id = 'h_content'> </hr> 
<p>
    Start the Secure Exchange Process:
    <ul>
        <li>Sign the Authorized Manufacturer Agreement with WPC</li>
        <li>Fill the Secure Exchange Process questionnaire</li>
        <li>
            Start a <a href="https://microchipsupport.force.com/s/"  target="_blank">support ticket</a> on microchipsupport.force.com 
            (attached the questionnaire output yaml file from your download folder) - provide a copy of the email informing WPC that 
            Microchip will be the MCSP (Manufacturer CA Service Provider)
        </li>
    </ul>
    Microchip provisioning team will provide a unique Custom Part Number per project and request WPC to sign the CSR (Certificate Signing Request).
</p>
<p>
Navigate to the ECC204 TFLXWPC configurator
<ul>
    <li>Select the WPC Authentication Use Case</li>
    <li>Populate the requested field</li>
    <li>Generate the Secure Exchange Package</li>
</ul>
</p>

<p>
    Once done, attach the generated files and attach it to the support case.
    Microchip will then enable the procurement of Verification Samples before starting mass-production.
</p>

</div>
</div>